In [13]:
import pandas as pd
hotel_rooms = pd.read_csv('room_type.csv')
hotel_rooms

,Expedia,Booking.com
0,"Deluxe Room, 1 King Bed",Deluxe King Room
1,"Standard Room, 1 King Bed, Accessible",Standard King Roll-in Shower Accessible
2,"Grand Corner King Room, 1 King Bed",Grand Corner King Room
3,"Suite, 1 King Bed (Parlor)",King Parlor Suite
4,"High-Floor Premium Room, 1 King Bed",High-Floor Premium King Room
...,...,...
98,"Room, 1 King Bed, Accessible, Resort View (Ali...",Alii Tower Resort View With King Bed - Mobilit...
99,"Room, 1 King Bed, Accessible, View (Rainbow, B...",Rainbow Tower Ocean View With King Bed - Mobil...
100,"Room, 1 King Bed, Ocean View (Alii)",Alii Tower Ocean View With King Bed
101,"Room, 1 King Bed, Oceanfront (Rainbow)",Rainbow Tower Ocean Front with King Bed


In [14]:
from fastfuzzymatch import FastFuzzyMatch
import rapidfuzz
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

ffm = FastFuzzyMatch(
    embedding_model=TfidfVectorizer(analyzer='char', ngram_range=(1, 4)),
    ##dimensionality_reduction_model=TruncatedSVD(n_components=100),
    clustering_model=NearestNeighbors(n_neighbors=5, metric='cosine', n_jobs=-1),
    fuzzy_model=rapidfuzz,
    fuzzy_scorer=rapidfuzz.fuzz.token_sort_ratio
)

test = ffm.find_matches(
    clean_df=hotel_rooms,
    clean_column='Booking.com',
    dirty_df=hotel_rooms,
    dirty_column='Expedia'
)


Fuzzy Matching Progress: 100%|██████████| 103/103 [00:00<00:00, 102348.57it/s]

Fuzzy matching completed in 0.040692806243896484 seconds


In [15]:
test

,Dirty,Clean,Ratio
0,"Deluxe Room, 1 King Bed",Deluxe Room - One King Bed,85.714286
1,"Standard Room, 1 King Bed, Accessible",Standard King Roll-in Shower Accessible,73.684211
2,"Grand Corner King Room, 1 King Bed",Grand Corner King Room,78.571429
3,"Suite, 1 King Bed (Parlor)",King Parlor Suite,55.813953
4,"High-Floor Premium Room, 1 King Bed",High-Floor Premium King Room,88.888889
...,...,...,...
98,"Room, 1 King Bed, Accessible, Resort View (Ali...",Alii Tower Resort View With King Bed - Mobilit...,58.823529
99,"Room, 1 King Bed, Accessible, View (Rainbow, B...",Rainbow Tower Ocean View With King Bed - Mobil...,49.572650
100,"Room, 1 King Bed, Ocean View (Alii)",Alii Tower Ocean View With King Bed,74.285714
101,"Room, 1 King Bed, Oceanfront (Rainbow)",Rainbow Tower Ocean Front with King Bed,46.753247
